In [2]:
import numpy as np
#import pandas as pd

import tensorflow.keras as keras
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import Image
from sklearn.decomposition import PCA

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from ultralytics import YOLO

In [2]:
train = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'training',
    seed = 50,
   
   # color_mode = 'grayscale'
    )

validation = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'validation',
    seed = 50,
   # class_names = ['Happy', 'Sad', 'Neutral'],
 #   color_mode = 'grayscale'
    )

Found 164 files belonging to 3 classes.
Using 132 files for training.


Found 164 files belonging to 3 classes.
Using 32 files for validation.


In [3]:
#convert to numpy images and then perform PCA to extract the values
#convert images to numpy
train_processed_numpy = train.as_numpy_iterator()
validation_processed_numpy = validation.as_numpy_iterator()

X_train, y_train = train_processed_numpy.next()
X_val, y_val = validation_processed_numpy.next()

In [4]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2] * X_train.shape[3]))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1] * X_val.shape[2] * X_val.shape[3]))

In [5]:
pca = PCA(n_components = 50)

X_train_lower_dim = pca.fit_transform(X_train)

In [6]:
pca.explained_variance_ratio_.sum()

#As can be seen from the output below, the first 50 components explain a substantial proportsion of the variance
#include train accuracy as well

0.94253206

In [7]:
#fitting an SVC object on this

svm = SVC()

svm.fit(X_train_lower_dim, np.argmax(y_train, axis = 1))

SVC()

In [8]:
#transforming the test data into the lower dimension
X_val_lower_dim = pca.transform(X_val)
y_pred = svm.predict(X_val_lower_dim)
y_pred_train = svm.predict(X_train_lower_dim)



In [9]:
print(accuracy_score(np.argmax(y_val, axis = 1),y_pred))
print(accuracy_score(np.argmax(y_train, axis = 1), y_pred_train))


0.46875
0.8333333333333334


# After face cropping

In [6]:
train = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'training',
    seed = 50,
   )

validation = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'validation',
    seed = 50,
  
    )

Found 164 files belonging to 3 classes.
Using 132 files for training.
Found 164 files belonging to 3 classes.
Using 32 files for validation.


In [7]:
#convert to numpy images and then perform PCA to extract the values
#convert images to numpy
train_processed_numpy = train.as_numpy_iterator()
validation_processed_numpy = validation.as_numpy_iterator()

X_train, y_train = train_processed_numpy.next()
X_val, y_val = validation_processed_numpy.next()

In [8]:
#convert the images to cropped faces
face_classifier = YOLO("../../app/src/face_detection/model/yolov8n-face.pt")

X_cropped_train = []
y_train_cropped = []

n_train = X_train.shape[0]
for i in range(n_train):
    face = X_train[i]
   
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face))
        pil_face = pil_face.resize((200, 200))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_cropped_train.append(numpy_cropped_face)
        y_train_cropped.append(y_train[i])

X_val_cropped = []
y_val_cropped = []
n_val = X_val.shape[0]

for i in range(n_val):
    face = X_val[i]
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face))
        pil_face = pil_face.resize((200, 200))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_val_cropped.append(numpy_cropped_face)
        y_val_cropped.append(y_val[i])


c:\Users\veena\OneDrive\Desktop\Nus\Y4S2\IS4242\Project Actual\moodify\emotion_detection\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 640x640 1 face, 208.8ms
Speed: 125.6ms preprocess, 208.8ms inference, 2716.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 116.0ms
Speed: 56.2ms preprocess, 116.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 125.5ms
Speed: 32.8ms preprocess, 125.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 145.6ms
Speed: 16.5ms preprocess, 145.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 140.5ms
Speed: 20.6ms preprocess, 140.5ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 167.0ms
Speed: 46.0ms preprocess, 167.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 157.8ms
Speed: 34.7ms preprocess, 157.8ms inference, 11.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 165.5ms
Speed: 23.4ms preprocess, 165.5ms inference, 1.0ms postprocess per image at sh

In [9]:
#change the variables to numpy arrays
X_cropped_train = np.array(X_cropped_train)

X_val_cropped = np.array(X_val_cropped)

print(X_cropped_train.shape)
print(X_val_cropped.shape)

#Flatten the RGB image into a flat tensor
X_cropped_train = X_cropped_train.reshape((X_cropped_train.shape[0], X_cropped_train.shape[1] * X_cropped_train.shape[2] * X_cropped_train.shape[3]))
X_val_cropped = X_val_cropped.reshape((X_val_cropped.shape[0], X_val_cropped.shape[1] * X_val_cropped.shape[2] * X_val_cropped.shape[3]))

(120, 200, 200, 3)
(29, 200, 200, 3)


In [10]:
y_train_lab = np.array(y_train_cropped).argmax(1)
y_val_lab = np.array(y_val_cropped).argmax(1)

In [11]:
pca = PCA(n_components = 50)

X_train_lower_dim = pca.fit_transform(X_cropped_train)

In [16]:
svc_cropped = SVC()

svc_cropped.fit(X_train_lower_dim, y_train_lab)

SVC()

In [17]:
X_val_lower_dim = pca.transform(X_val_cropped)

In [18]:
y_pred = svc_cropped.predict(X_val_lower_dim)
y_pred_train = svc_cropped.predict(X_train_lower_dim)
#display the accuracy of the model
print(accuracy_score(y_val_lab, y_pred))
#note that the accuracy is lower compared to not using the cropped image
print(accuracy_score(y_train_lab, y_pred_train))

0.41379310344827586
0.8333333333333334
